In [1]:
import pandas as pd
import os
import requests, zipfile, io
import numpy as np


# Financial Data

In [16]:
# import financials -- all num.txt and sub.txt
years = list(range(2009,2021))
quarters = ['q1','q2','q3','q4']
filenames = ['num.txt','sub.txt']   
dataset = 'financial-statement-data-sets'
# 'pre.txt','tag.txt

def SEC_data(dataset,quarter,year,filename):
    url = f"https://www.sec.gov/files/dera/data/{dataset}/{year}{quarter}.zip"
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall()
    df = pd.read_csv(filename,sep="\t",dtype={"cik":str})
    return df

In [17]:
financials = []    
for year in years:
    for quarter in quarters:
        for filename in filenames:
            df = SEC_data(dataset,quarter, year, filename)
            financials.append(df)

/Users/lorelablaka/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/lorelablaka/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3357: DtypeWarning: Columns (35) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [18]:
numi = []
subi = []
financials_all = []

for n in list(range(len(financials))):
    if n % 2 == 0:
        numi.append(n)
    else:  
        subi.append(n)
        
for n, s in zip(numi,subi):
    merged_df = financials[n].merge(financials[s][["adsh","cik","name","period","fy","fp","filed", "form","instance"]])
    financials_all.append(merged_df)

Filter for only 10-Ks only
<br> Companies have different FYs and larger companies take much longer to submit filings -- will not filter for quarter 

In [19]:
financials = pd.concat(financials_all)
financials.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 105850078 entries, 0 to 2373223
Data columns (total 17 columns):
 #   Column    Dtype  
---  ------    -----  
 0   tag       object 
 1   version   object 
 2   coreg     object 
 3   ddate     object 
 4   qtrs      object 
 5   uom       object 
 6   value     float64
 7   footnote  object 
 8   adsh      object 
 9   cik       object 
 10  name      object 
 11  period    float64
 12  fy        float64
 13  fp        object 
 14  filed     object 
 15  form      object 
 16  instance  object 
dtypes: float64(3), object(14)
memory usage: 14.2+ GB


In [20]:
tags = pd.DataFrame(financials['tag'].value_counts())
tags[tags['tag']>=500000]

,tag
NetIncomeLoss,1219874
CashAndCashEquivalentsAtCarryingValue,1090750
OperatingIncomeLoss,905307
IncomeTaxExpenseBenefit,853246
StockholdersEquity,795342
Assets,698393
InterestExpense,673430
LiabilitiesAndStockholdersEquity,665961
EarningsPerShareBasic,659969
EarningsPerShareDiluted,639670


In [21]:
financials_tags = financials[financials["tag"].isin(["NetIncomeLoss",
                                                 "CashAndCashEquivalentsAtCarryingValue",
                                                 "OperatingIncomeLoss",
                                                 "StockholdersEquity",
                                                 "Assets",
                                                 
                                                 "LiabilitiesAndStockholdersEquity",
                                                 "LiabilitiesAndStockholdersEquity",
                                                 
                                                 "EarningsPerShareBasic",
                                                 "EarningsPerShareDiluted",
                                                 
                                                 "NetCashProvidedByUsedInOperatingActivities",
                                                 "NetCashProvidedByUsedInFinancingActivities",
                                                 
                                                 "RetainedEarningsAccumulatedDeficit",
                                                 "EarningsPerShareDiluted",
                                                 "CommonStockSharesAuthorized"
                                                 ])]

In [22]:
annual_raw = financials_tags[financials_tags["form"].isin(["10-K","20-K","40-K"])]


In [23]:
annual_raw.drop(annual_raw.columns[np.r_[7]], axis=1, inplace=True)


/Users/lorelablaka/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [24]:
annual_raw.drop_duplicates(inplace= True)

<ipython-input-24-f35c028b3ecf>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  annual_raw.drop_duplicates(inplace= True)


In [25]:
annual_raw.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2393312 entries, 3481 to 2288717
Data columns (total 16 columns):
 #   Column    Dtype  
---  ------    -----  
 0   tag       object 
 1   version   object 
 2   coreg     object 
 3   ddate     object 
 4   qtrs      object 
 5   uom       object 
 6   value     float64
 7   adsh      object 
 8   cik       object 
 9   name      object 
 10  period    float64
 11  fy        float64
 12  fp        object 
 13  filed     object 
 14  form      object 
 15  instance  object 
dtypes: float64(3), object(13)
memory usage: 310.4+ MB


In [198]:
annual = annual_raw.pivot_table(values='value', 
                          index= ['version',
                                  'coreg',
                                  'ddate',
                                  'qtrs',
                                  'uom',
                                  'adsh',
                                  'cik',
                                  'name',
                                   'filed',
                                  'form',
                                  'instance'],
                          columns = ['tag','fy'])


In [199]:
annual.reset_index(inplace=True)

In [201]:
annual

tag          version                      coreg     ddate qtrs  uom  \
fy                                                                    
0       us-gaap/2009                   AllOther  20081231    0  USD   
1       us-gaap/2009                   AllOther  20091231    0  USD   
2       us-gaap/2009                   AllOther  20101231    0  USD   
3       us-gaap/2009        AllOtherSubsidaries  20081231    0  USD   
4       us-gaap/2009        AllOtherSubsidaries  20081231    4  USD   
...              ...                        ...       ...  ...  ...   
114470  us-gaap/2020                    Captive  20200930    0  USD   
114471  us-gaap/2020  GilatSatelliteNetworksLtd  20200630    0  USD   
114472  us-gaap/2020  GilatSatelliteNetworksLtd  20200630    2  USD   
114473  us-gaap/2020            ICEEDistributor  20200930    4  USD   
114474  us-gaap/2020            RMXResourcesLLC  20191231    0  USD   

tag                     adsh      cik                                  name  \
fy                                                                            
0       0000092122-11-000013    92122                           SOUTHERN CO   
1       0000092122-11-000013    92122                           SOUTHERN CO   
2       0000092122-11-000013    92122                           SOUTHERN CO   
3       0000950123-11-019900     6769                           APACHE CORP   
4       0000950123-11-019900     6769                           APACHE CORP   
...                      ...      ...                                   ...   
114470  0001584509-20-000230  1584509                               ARAMARK   
114471  0000023197-20-000127    23197  COMTECH TELECOMMUNICATIONS CORP /DE/   
114472  0000023197-20-000127    23197  COMTECH TELECOMMUNICATIONS CORP /DE/   
114473  0001437749-20-024447   785956                  J&J SNACK FOODS CORP   
114474  0001185185-20-000394  1694617                   ROYALE ENERGY, INC.   

tag        filed  form  ... StockholdersEquity                              \
fy                      ...             2011.0 2012.0 2013.0 2014.0 2015.0   
0       20110225  10-K  ...                NaN    NaN    NaN    NaN    NaN   
1       20110225  10-K  ...                NaN    NaN    NaN    NaN    NaN   
2       20110225  10-K  ...                NaN    NaN    NaN    NaN    NaN   
3       20110228  10-K  ...                NaN    NaN    NaN    NaN    NaN   
4       20110228  10-K  ...                NaN    NaN    NaN    NaN    NaN   
...          ...   ...  ...                ...    ...    ...    ...    ...   
114470  20201124  10-K  ...                NaN    NaN    NaN    NaN    NaN   
114471  20200929  10-K  ...                NaN    NaN    NaN    NaN    NaN   
114472  20200929  10-K  ...                NaN    NaN    NaN    NaN    NaN   
114473  20201125  10-K  ...                NaN    NaN    NaN    NaN    NaN   
114474  20200330  10-K  ...                NaN    NaN    NaN    NaN    NaN   

tag                                        
fy     2016.0 2017.0 2018.0 2019.0 2020.0  
0         NaN    NaN    NaN    NaN    NaN  
1         NaN    NaN    NaN    NaN    NaN  
2         NaN    NaN    NaN    NaN    NaN  
3         NaN    NaN    NaN    NaN    NaN  
4         NaN    NaN    NaN    NaN    NaN  
...       ...    ...    ...    ...    ...  
114470    NaN    NaN    NaN    NaN    NaN  
114471    NaN    NaN    NaN    NaN    NaN  
114472    NaN    NaN    NaN    NaN    NaN  
114473    NaN    NaN    NaN    NaN    NaN  
114474    NaN    NaN    NaN    NaN    NaN  

[114475 rows x 155 columns]

In [202]:
pd.set_option('display.max_columns', None)


In [205]:
annual.describe()

tag           ddate           qtrs         filed        Assets                \
fy                                                      2009.0        2010.0   
count  1.144750e+05  114475.000000  1.144750e+05  4.600000e+01  7.510000e+02   
mean   2.012904e+07       1.660258  2.014773e+07  1.021509e+11  2.764048e+10   
std    2.297542e+04       1.915882  2.055928e+04  2.721153e+11  1.491281e+11   
min    2.002063e+07       0.000000  2.010020e+07  2.689000e+09 -1.223592e+12   
25%    2.011123e+07       0.000000  2.013032e+07  7.151849e+09  2.429576e+09   
50%    2.013093e+07       0.000000  2.015021e+07  1.008329e+10  8.254500e+09   
75%    2.014123e+07       4.000000  2.016031e+07  2.993925e+10  1.784180e+10   
max    2.020093e+07      23.000000  2.020122e+07  1.227040e+12  1.231372e+12   

tag                                                                          \
fy           2011.0        2012.0        2013.0        2014.0        2015.0   
count  3.046000e+03  6.342000e+03  5.832000e+03  5.474000e+03  4.928000e+03   
mean   8.373742e+09  4.505217e+09  5.610764e+09  6.133465e+09  9.359284e+09   
std    7.512206e+10  4.680468e+10  5.074296e+10  4.941152e+10  9.270881e+10   
min   -1.267076e+12 -2.741300e+11 -2.235260e+11 -2.045840e+11 -1.765670e+11   
25%    2.217315e+08  5.154258e+07  6.662096e+07  8.549700e+07  1.117305e+08   
50%    1.828846e+09  5.638980e+08  7.351835e+08  8.613100e+08  9.967210e+08   
75%    5.789970e+09  2.945459e+09  3.437301e+09  4.027637e+09  4.484526e+09   
max    1.291288e+12  2.209974e+12  2.209974e+12  2.104534e+12  2.850000e+12   

tag                                                                          \
fy           2016.0        2017.0        2018.0        2019.0        2020.0   
count  4.017000e+03  1.234000e+03  7.120000e+02  6.330000e+02  2.900000e+01   
mean   9.761133e+09  1.086097e+10  1.057581e+10  1.252469e+10  1.169525e+09   
std    5.456991e+10  3.280181e+10  2.554415e+10  2.678605e+10  1.703762e+09   
min   -1.246760e+11 -2.929710e+10 -2.469130e+10 -2.108890e+10  0.000000e+00   
25%    2.108258e+08  1.078745e+08  5.925375e+07  1.114410e+08  8.588480e+05   
50%    1.389190e+09  1.960904e+09  2.255018e+09  3.413000e+09  1.240300e+08   
75%    5.731790e+09  7.864250e+09  9.293075e+09  1.146900e+10  1.743600e+09   
max    1.722480e+12  4.799110e+11  2.197970e+11  1.891520e+11  5.291200e+09   

tag   CashAndCashEquivalentsAtCarryingValue                              \
fy                                   2009.0        2010.0        2011.0   
count                          8.600000e+01  1.036000e+03  4.479000e+03   
mean                           1.878099e+09  7.722515e+08  2.790234e+08   
std                            8.128880e+09  3.961329e+09  2.030167e+09   
min                            5.000000e+06 -5.150000e+09 -5.150000e+09   
25%                            5.050000e+07  3.660000e+06  5.625000e+05   
50%                            2.060000e+08  7.371800e+07  1.910000e+07   
75%                            3.802500e+08  3.878562e+08  1.440000e+08   
max                            6.435600e+10  6.258400e+10  7.670200e+10   

tag                                                                          \
fy           2012.0        2013.0        2014.0        2015.0        2016.0   
count  8.938000e+03  8.875000e+03  8.508000e+03  7.826000e+03  6.317000e+03   
mean   1.403676e+08  1.514862e+08  1.803293e+08  1.958889e+08  2.234908e+08   
std    5.946722e+08  1.009838e+09  2.144769e+09  2.365816e+09  2.288232e+09   
min   -9.491000e+08 -1.016000e+09 -3.714000e+09 -3.714000e+09 -1.500000e+08   
25%    2.992508e+05  1.524545e+05  1.580000e+05  2.000000e+05  9.930000e+05   
50%    8.335000e+06  7.216000e+06  7.031000e+06  7.140500e+06  1.180400e+07   
75%    6.550000e+07  6.319000e+07  6.278125e+07  6.121875e+07  7.563700e+07   
max    1.679900e+10  6.414422e+10  1.249910e+11  1.028180e+11  1.003040e+11   

tag                                                            \
fy        

# Crowdfunding Data

In [29]:
# import crowdfunding -- 'FORM_C_SUBMISSION.tsv','FORM_C_DISCLOSURE.tsv'
quarters = ['q1_cf','q2_cf','q3_cf','q4_cf']
years = list(range(2017,2021))
dataset = 'crowdfunding-offerings-data-sets'

crowdfunding = []    
for year in years:
    for quarter in quarters:
        file_disc = f"{year}{quarter.capitalize()}/FORM_C_DISCLOSURE.tsv"
        file_issuer = f"{year}{quarter.capitalize()}/FORM_C_ISSUER_INFORMATION.tsv"        
        filenames =[file_disc,file_issuer]
        for filename in filenames:
            df = SEC_data(dataset, quarter, year, filename)
            crowdfunding.append(df)
 

In [30]:
disc_i = []
issuer_i = []
crowdfunding_all = []

for n in list(range(len(crowdfunding))):
    if n % 2 == 0:
        disc_i.append(n)
    else:  
        issuer_i.append(n)
        
for n, s in zip(disc_i,issuer_i):
    df = crowdfunding[n].merge(crowdfunding[s])
    crowdfunding_all.append(df)

In [31]:
crowdfunding = pd.concat(crowdfunding_all)

In [32]:
crowdfunding.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9553 entries, 0 to 1013
Data columns (total 51 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ACCESSION_NUMBER                9553 non-null   object 
 1   COMPENSATIONAMOUNT              7958 non-null   object 
 2   FINANCIALINTEREST               5716 non-null   object 
 3   SECURITYOFFEREDTYPE             7958 non-null   object 
 4   SECURITYOFFEREDOTHERDESC        3625 non-null   object 
 5   NOOFSECURITYOFFERED             6401 non-null   float64
 6   PRICE                           7300 non-null   float64
 7   PRICEDETERMINATIONMETHOD        4245 non-null   object 
 8   OFFERINGAMOUNT                  7958 non-null   float64
 9   OVERSUBSCRIPTIONACCEPTED        7958 non-null   object 
 10  OVERSUBSCRIPTIONALLOCATIONTYPE  7875 non-null   object 
 11  DESCOVERSUBSCRIPTION            4575 non-null   object 
 12  MAXIMUMOFFERINGAMOUNT           78

In [33]:
crowdfunding.drop(crowdfunding.columns[np.r_[30:45,46,48,49]], axis=1, inplace=True)

In [34]:
crowdfunding.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9553 entries, 0 to 1013
Data columns (total 33 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ACCESSION_NUMBER                9553 non-null   object 
 1   COMPENSATIONAMOUNT              7958 non-null   object 
 2   FINANCIALINTEREST               5716 non-null   object 
 3   SECURITYOFFEREDTYPE             7958 non-null   object 
 4   SECURITYOFFEREDOTHERDESC        3625 non-null   object 
 5   NOOFSECURITYOFFERED             6401 non-null   float64
 6   PRICE                           7300 non-null   float64
 7   PRICEDETERMINATIONMETHOD        4245 non-null   object 
 8   OFFERINGAMOUNT                  7958 non-null   float64
 9   OVERSUBSCRIPTIONACCEPTED        7958 non-null   object 
 10  OVERSUBSCRIPTIONALLOCATIONTYPE  7875 non-null   object 
 11  DESCOVERSUBSCRIPTION            4575 non-null   object 
 12  MAXIMUMOFFERINGAMOUNT           78

In [35]:
crowdfunding.rename(columns={
    'ACCESSION_NUMBER':'accession_number',
    'COMPENSATIONAMOUNT': 'compensation_amount',
    'FINANCIALINTEREST': 'financial_interest',
    'SECURITYOFFEREDTYPE': 'security_offered_type',
    'SECURITYOFFEREDOTHERDESC': 'security_offered_other_desc',
    'NOOFSECURITYOFFERED':'no_of_security_offered', 
    'PRICE': 'price',
    'PRICEDETERMINATIONMETHOD': 'price_determination_method',
    'OFFERINGAMOUNT': 'offering_amount',
    'DESCOVERSUBSCRIPTION' : 'desc_over_subscription',
   
    'OVERSUBSCRIPTIONACCEPTED': 'oversubscription_accepted',    
    'OVERSUBSCRIPTIONALLOCATIONTYPE':'oversubscriptional_location_type', 
    'COMPENSATIONAMOUNT': 'des_cover_subscription',
    'MAXIMUMOFFERINGAMOUNT': 'maximum_offering_amount',
    'DEADLINEDATE': 'deadline_date',
    
    'CURRENTEMPLOYEES': 'current_employees',
    
    
    'TOTALASSETMOSTRECENTFISCALYEAR':'total_asset_most_recent_fiscalyear', 
    'TOTALASSETPRIORFISCALYEAR': 'total_asset_prior_fiscal_year',
    
    'CASHEQUIMOSTRECENTFISCALYEAR': 'cash_equi_most_recent_fiscal_year',
    'CASHEQUIPRIORFISCALYEAR': 'cash_equi_prior_fiscal_year',
    
    'ACTRECEIVEDRECENTFISCALYEAR': 'act_received_recent_fiscal_year',
    'ACTRECEIVEDPRIORFISCALYEAR':'act_received_prior_fiscal_year', 
    
    'SHORTTERMDEBTMRECENTFISCALYEAR': 'short_term_debt_m_recent_fiscal_year',
    'SHORTTERMDEBTPRIORFISCALYEAR': 'short_term_debt_prior_fiscal_year',
    
    'LONGTERMDEBTRECENTFISCALYEAR': 'long_term_debt_recent_fiscal_year',
    'LONGTERMDEBTPRIORFISCALYEAR': 'long_term_debt_prior_fiscal_year',    
    
    'REVENUEMOSTRECENTFISCALYEAR':'revenue_most_recent_fiscal_year', 
    'REVENUEPRIORFISCALYEAR': 'revenue_prior_fiscal_year',
    
    'COSTGOODSSOLDRECENTFISCALYEAR': 'cost_goods_sold_recent_fiscal_year',
    'COSTGOODSSOLDPRIORFISCALYEAR': 'cost_goods_sold_prior_fiscal_year',
    
    'TAXPAIDMOSTRECENTFISCALYEAR': 'tax_paid_most_recent_fiscal_year',
    
    'ZIPCODE':'zipcode', 
    'COMPANYNAME': 'company_name',
    'CRDNUMBER': 'financial_interest'
},
                inplace=True)

In [36]:
crowdfunding.columns

Index(['accession_number', 'des_cover_subscription', 'financial_interest',
       'security_offered_type', 'security_offered_other_desc',
       'no_of_security_offered', 'price', 'price_determination_method',
       'offering_amount', 'oversubscription_accepted',
       'oversubscriptional_location_type', 'desc_over_subscription',
       'maximum_offering_amount', 'deadline_date', 'current_employees',
       'total_asset_most_recent_fiscalyear', 'total_asset_prior_fiscal_year',
       'cash_equi_most_recent_fiscal_year', 'cash_equi_prior_fiscal_year',
       'act_received_recent_fiscal_year', 'act_received_prior_fiscal_year',
       'short_term_debt_m_recent_fiscal_year',
       'short_term_debt_prior_fiscal_year',
       'long_term_debt_recent_fiscal_year', 'long_term_debt_prior_fiscal_year',
       'revenue_most_recent_fiscal_year', 'revenue_prior_fiscal_year',
       'cost_goods_sold_recent_fiscal_year',
       'cost_goods_sold_prior_fiscal_year', 'tax_paid_most_recent_fiscal_year'

# Save Collected Data 

In [39]:
import joblib 
joblib.dump(crowdfunding,"crowdfunding.pkl")
joblib.dump(annual,"sec_10k_raw.pkl")

['age.pkl']

In [40]:
ls


2017Q1_cf/             2019Q3_cf/             data/
2017Q2_cf/             2019Q4_cf/             data_collection.ipynb
2017Q3_cf/             2020Q1_cf/             data_merge.ipynb
2017Q4_cf/             2020Q2_cf/             managers.pkl
2018Q1_cf/             2020Q3_cf/             managers_melt.pkl
2018Q2_cf/             2020Q4_cf/             num.txt
2018Q3_cf/             2021Q1_cf/             pre.txt
2018Q4_cf/             age.pkl                readme.htm
2019Q1_cf/             annual.pkl             sub.txt
2019Q2_cf/             crowdfunding.pkl       tag.txt
